<a href="https://colab.research.google.com/github/rjrizani/Chrome-Extension/blob/master/first_steps_with_tensor_flow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Copyright 2017 Google LLC.

In [1]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Langkah Pertama dengan TensorFlow

**Tujuan Pembelajaran:**
  * Mempelajari konsep TensorFlow dasar
  * Menggunakan kelas `LinearRegressor` di TensorFlow untuk memprediksi median harga perumahan, pada perincian blok kota, berdasarkan satu fitur masukan
  * Mengevaluasi akurasi prediksi model menggunakan Akar Rataan Kuadrat Galat (ARKG)
  * Meningkatkan akurasi model dengan menyesuaikan hyperparameter-nya

[Data](https://developers.google.com/machine-learning/crash-course/california-housing-data-description) didasarkan pada data sensus tahun 1990 dari California.

## Penyiapan

Di sel pertama ini, kita akan memuat library yang diperlukan.

In [ ]:
from __future__ import print_function

import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
%tensorflow_version 1.x
import tensorflow as tf
from tensorflow.python.data import Dataset

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

Berikutnya, kita akan memuat kumpulan data.

In [ ]:
california_housing_dataframe = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv", sep=",")

Kita akan mengacak data tersebut, namun pastikan data tidak terkena efek pengurutan patologis yang dapat mengganggu performa Penurunan Gradien Stokastik. Selain itu, kita akan menskalakan `median_house_value` dalam satuan ribuan, sehingga dapat dipelajari sedikit lebih mudah dengan kecepatan pembelajaran dalam kisaran yang biasanya digunakan.

In [ ]:
california_housing_dataframe = california_housing_dataframe.reindex(
    np.random.permutation(california_housing_dataframe.index))
california_housing_dataframe["median_house_value"] /= 1000.0
california_housing_dataframe

## Periksa Data

Ini adalah ide yang bagus untuk sedikit memahami data Anda sebelum menggunakannya.

Kita akan mencetak ringkasan singkat dari beberapa statistik yang berguna di setiap kolom: jumlah contoh, rataan, simpangan baku, nilai maks, nilai min, dan berbagai kuantil.

In [ ]:
california_housing_dataframe.describe()

## Buat Model Pertama

Dalam latihan ini, kita akan mencoba memprediksi `median_house_value`, yang akan menjadi label (terkadang juga disebut target). Kita akan menggunakan `total_rooms` sebagai fitur masukan kita.

**CATATAN:** Data kita berada di tingkat blok kota, sehingga fitur ini merepresentasikan jumlah total ruangan di blok tersebut.

Untuk melatih model, kita akan menggunakan antarmuka [LinearRegressor](https://www.tensorflow.org/api_docs/python/tf/estimator/LinearRegressor) yang diberikan oleh TensorFlow [Estimator](https://www.tensorflow.org/get_started/estimator) API. API ini menangani banyak sistem model tingkat rendah, dan menampilkan metode yang nyaman untuk melakukan inferensi, evaluasi, dan pelatihan model.

### Langkah 1: Tentukan Fitur dan Konfigurasikan Kolom Fitur

Untuk mengimpor data pelatihan ke TensorFlow, kita perlu menentukan jenis data apa yang ada dalam setiap fitur. Ada dua jenis data utama yang akan kita gunakan dalam latihan ini dan latihan berikutnya:

* **Data Kategorikal**: Data yang bersifat tekstual. Dalam latihan ini, kumpulan data perumahan kita tidak berisi fitur kategoris apa pun, tetapi contoh yang mungkin Anda lihat adalah gaya rumah dan kata dalam iklan real estat.

* **Data Numerik**: Data yang berupa bilangan (bulat atau pecahan) dan data yang ingin Anda perlakukan sebagai bilangan. Seperti yang akan kita bahas lebih lanjut, terkadang Anda mungkin ingin memperlakukan data numerik (misalnya kode pos) selayaknya data kategorikal.

Di TensorFlow, kita menunjukkan jenis data fitur menggunakan konstruksi yang disebut **kolom fitur**. Kolom fitur hanya menyimpan deskripsi data fitur dan tidak berisi data fitur itu sendiri.

Untuk memulainya, kita hanya akan menggunakan satu fitur masukan numerik, yaitu `total_rooms`. Kode berikut mengambil data `total_rooms` dari `california_housing_dataframe` dan mendefinisikan kolom fitur menggunakan `numeric_column`, yang menentukan datanya berupa angka:

In [ ]:
# Define the input feature: total_rooms.
my_feature = california_housing_dataframe[["total_rooms"]]

# Configure a numeric feature column for total_rooms.
feature_columns = [tf.feature_column.numeric_column("total_rooms")]

**CATATAN:** Bentuk data `total_rooms` kita adalah array satu dimensi (daftar jumlah total ruangan untuk setiap blok). Ini adalah bentuk default untuk `numeric_column`, sehingga kita tidak harus meneruskannya sebagai argumen.

### Langkah 2: Tentukan Target

Selanjutnya, kita akan menentukan target, yaitu `median_house_value`. Sekali lagi, kita dapat mengambilnya dari `california_housing_dataframe`:

In [ ]:
# Define the label.
targets = california_housing_dataframe["median_house_value"]

### Langkah 3: Konfigurasikan LinearRegressor

Selanjutnya, kita akan mengonfigurasi model regresi linear menggunakan LinearRegressor. Kita akan melatih model ini menggunakan `GradientDescentOptimizer`, yang menerapkan Tumpukan Mini Penurunan Gradien Stokastik (PGS). Argumen `learning_rate` mengontrol ukuran langkah gradien.

**CATATAN:** Agar aman, kita juga menerapkan [pemotongan gradien](https://developers.google.com/machine-learning/glossary/#gradient_clipping) ke pengoptimal menggunakan `clip_gradients_by_norm`. Pemotongan gradien memastikan ukuran gradien tidak terlalu besar selama pelatihan, yang dapat menyebabkan kegagalan pada penurunan gradien. 

In [ ]:
# Use gradient descent as the optimizer for training the model.
my_optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.0000001)
my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)

# Configure the linear regression model with our feature columns and optimizer.
# Set a learning rate of 0.0000001 for Gradient Descent.
linear_regressor = tf.estimator.LinearRegressor(
    feature_columns=feature_columns,
    optimizer=my_optimizer
)

### Langkah 4: Tentukan Fungsi Masukan

Untuk mengimpor data perumahan di California ke `LinearRegressor`, kita perlu mendefinisikan fungsi masukan, yang menginstruksikan TensorFlow bagaimana cara melakukan pra-pemrosesan terhadap data, serta cara menumpuk, mengacak, dan mengulanginya selama pelatihan model.

Pertama, kita akan mengonversi data fitur *pandas* menjadi kamus array NumPy. Kemudian, kita dapat menggunakan [API Kumpulan Data](https://www.tensorflow.org/programmers_guide/datasets) TensorFlow untuk membuat objek kumpulan data dari data tersebut, lalu membaginya menjadi tumpukan `batch_size`, yang akan diulang untuk jumlah iterasi pelatihan (num_epochs) yang ditentukan.

**CATATAN:** Ketika nilai default `num_epochs=None` diteruskan ke `repeat()`, data masukan akan diulang tanpa batas.

Berikutnya, jika `shuffle` disetel ke `True`, kita akan mengacak data sehingga diteruskan ke model secara acak selama pelatihan. Argumen `buffer_size` menentukan ukuran kumpulan data tempat `shuffle` akan mengambil sampel secara acak.

Terakhir, fungsi masukan kita akan membuat iterator untuk kumpulan data dan mengembalikan tumpukan data berikutnya ke LinearRegressor.

In [ ]:
def my_input_fn(features, targets, batch_size=1, shuffle=True, num_epochs=None):
    """Trains a linear regression model of one feature.
  
    Args:
      features: pandas DataFrame of features
      targets: pandas DataFrame of targets
      batch_size: Size of batches to be passed to the model
      shuffle: True or False. Whether to shuffle the data.
      num_epochs: Number of epochs for which data should be repeated. None = repeat indefinitely
    Returns:
      Tuple of (features, labels) for next data batch
    """
  
    # Convert pandas data into a dict of np arrays.
    features = {key:np.array(value) for key,value in dict(features).items()}                                           
 
    # Construct a dataset, and configure batching/repeating.
    ds = Dataset.from_tensor_slices((features,targets)) # warning: 2GB limit
    ds = ds.batch(batch_size).repeat(num_epochs)
    
    # Shuffle the data, if specified.
    if shuffle:
      ds = ds.shuffle(buffer_size=10000)
    
    # Return the next batch of data.
    features, labels = ds.make_one_shot_iterator().get_next()
    return features, labels

**CATATAN:** Kita akan terus menggunakan fungsi masukan yang sama ini di latihan selanjutnya. Untuk dokumentasi mendetail tentang fungsi masukan dan `Dataset` API, lihat [Panduan Pemrogram TensorFlow](https://www.tensorflow.org/programmers_guide/datasets).

### Langkah 5: Latih Model

Kita sekarang dapat memanggil `train()` pada `linear_regressor` untuk melatih model. Kita akan menggabungkan `my_input_fn` dalam `lambda` sehingga kita dapat meneruskannya dalam `my_feature` dan `target` sebagai argumen (lihat [tutorial fungsi masukan TensorFlow](https://www.tensorflow.org/get_started/input_fn#passing_input_fn_data_to_your_model) ini untuk detail selengkapnya), dan untuk memulainya, kita akan melatihnya 100 langkah.

In [ ]:
_ = linear_regressor.train(
    input_fn = lambda:my_input_fn(my_feature, targets),
    steps=100
)

### Langkah 6: Evaluasi Model

Mari buat prediksi tentang data pelatihan tersebut, untuk melihat seberapa baik model menyesuaikannya selama pelatihan.

**CATATAN:** Error pelatihan mengukur seberapa baik model Anda menyesuaikan dengan data pelatihan, tetapi **_tidak_** mengukur seberapa baik model Anda **_menggeneralisasi ke data baru_**. Dalam latihan selanjutnya, Anda akan mempelajari cara membagi data untuk mengevaluasi kemampuan model untuk melakukan generalisasi.


In [ ]:
# Create an input function for predictions.
# Note: Since we're making just one prediction for each example, we don't 
# need to repeat or shuffle the data here.
prediction_input_fn =lambda: my_input_fn(my_feature, targets, num_epochs=1, shuffle=False)

# Call predict() on the linear_regressor to make predictions.
predictions = linear_regressor.predict(input_fn=prediction_input_fn)

# Format predictions as a NumPy array, so we can calculate error metrics.
predictions = np.array([item['predictions'][0] for item in predictions])

# Print Mean Squared Error and Root Mean Squared Error.
mean_squared_error = metrics.mean_squared_error(predictions, targets)
root_mean_squared_error = math.sqrt(mean_squared_error)
print("Mean Squared Error (on training data): %0.3f" % mean_squared_error)
print("Root Mean Squared Error (on training data): %0.3f" % root_mean_squared_error)

Apakah model ini sudah baik? Bagaimana Anda akan menilai seberapa besar error ini?

Rataan Kuadrat Galat (RKG) bisa sulit diinterpretasikan, sehingga kita sering menggunakan Akar Rataan Kuadrat Galat (ARKG) sebagai gantinya. Kelebihan ARKG adalah dapat diinterpretasikan pada skala yang sama dengan target aslinya.

Mari bandingkan ARKG dengan perbedaan nilai min dan maks dari target kita:

In [ ]:
min_house_value = california_housing_dataframe["median_house_value"].min()
max_house_value = california_housing_dataframe["median_house_value"].max()
min_max_difference = max_house_value - min_house_value

print("Min. Median House Value: %0.3f" % min_house_value)
print("Max. Median House Value: %0.3f" % max_house_value)
print("Difference between Min. and Max.: %0.3f" % min_max_difference)
print("Root Mean Squared Error: %0.3f" % root_mean_squared_error)

Error kita mencakup hampir setengah dari kisaran nilai target. Bisakah kita melakukannya dengan lebih baik?

Pertanyaan ini sering diajukan kepada setiap developer model. Mari kembangkan beberapa strategi dasar untuk mengurangi error model.

Langkah pertama yang dapat kita lakukan adalah melihat seberapa baik prediksi kita menyesuaikan dengan target, dalam hal statistik ringkasan secara keseluruhan.

In [ ]:
calibration_data = pd.DataFrame()
calibration_data["predictions"] = pd.Series(predictions)
calibration_data["targets"] = pd.Series(targets)
calibration_data.describe()

Oke, mungkin informasi ini bermanfaat. Bagaimana nilai rataan jika dibandingkan dengan ARKG model? Bagaimana dengan kuantil yang beragam?

Kita juga dapat menggambarkan data dan garis yang telah dipelajari. Perlu diingat bahwa regresi linear pada satu fitur dapat diambil sebagai masukan pemetaan garis *x* ke keluaran *y*.

Pertama, kita akan mendapatkan sampel acak yang seragam dari data, sehingga kita dapat membuat bagan sebar yang bisa dibaca.

In [ ]:
sample = california_housing_dataframe.sample(n=300)

Selanjutnya, kita akan menggambarkan garis yang telah dipelajari, menggambar dari bobot fitur dan istilah bias model, bersama dengan bagan sebar. Garisnya akan ditunjukkan dengan warna merah.

In [ ]:
# Get the min and max total_rooms values.
x_0 = sample["total_rooms"].min()
x_1 = sample["total_rooms"].max()

# Retrieve the final weight and bias generated during training.
weight = linear_regressor.get_variable_value('linear/linear_model/total_rooms/weights')[0]
bias = linear_regressor.get_variable_value('linear/linear_model/bias_weights')

# Get the predicted median_house_values for the min and max total_rooms values.
y_0 = weight * x_0 + bias 
y_1 = weight * x_1 + bias

# Plot our regression line from (x_0, y_0) to (x_1, y_1).
plt.plot([x_0, x_1], [y_0, y_1], c='r')

# Label the graph axes.
plt.ylabel("median_house_value")
plt.xlabel("total_rooms")

# Plot a scatter plot from our data sample.
plt.scatter(sample["total_rooms"], sample["median_house_value"])

# Display graph.
plt.show()

Baris awal ini terlihat terlalu jauh. Lihat apakah Anda dapat melihat kembali statistik ringkasan dan melihat informasi yang sama yang dienkode di sana.

Bersama, pemeriksaan awal ini menyarankan bahwa kita mungkin dapat menemukan garis yang jauh lebih baik.

## Sesuaikan Hyperparameter Model
Untuk latihan ini, kita telah menempatkan semua kode di atas dalam satu fungsi untuk kenyamanan. Anda dapat memanggil fungsi dengan parameter yang berbeda untuk melihat efeknya.

Dalam fungsi ini, kita akan memproses dalam 10 periode yang terbagi secara merata sehingga kita dapat mengamati peningkatan model pada setiap periode.

Untuk setiap periode, kita menghitung dan membuat grafik kerugian pelatihan. Tindakan ini dapat membantu Anda menilai saat sebuah model dikonvergensi, atau jika model perlu lebih banyak iterasi.

Kita juga akan menggambar bobot fitur dan nilai bias yang dipelajari oleh model dari waktu ke waktu. Tindakan ini adalah cara alternatif untuk melihat bagaimana model dikonvergensi.

In [ ]:
def train_model(learning_rate, steps, batch_size, input_feature="total_rooms"):
  """Trains a linear regression model of one feature.
  
  Args:
    learning_rate: A `float`, the learning rate.
    steps: A non-zero `int`, the total number of training steps. A training step
      consists of a forward and backward pass using a single batch.
    batch_size: A non-zero `int`, the batch size.
    input_feature: A `string` specifying a column from `california_housing_dataframe`
      to use as input feature.
  """
  
  periods = 10
  steps_per_period = steps / periods

  my_feature = input_feature
  my_feature_data = california_housing_dataframe[[my_feature]]
  my_label = "median_house_value"
  targets = california_housing_dataframe[my_label]

  # Create feature columns.
  feature_columns = [tf.feature_column.numeric_column(my_feature)]
  
  # Create input functions.
  training_input_fn = lambda:my_input_fn(my_feature_data, targets, batch_size=batch_size)
  prediction_input_fn = lambda: my_input_fn(my_feature_data, targets, num_epochs=1, shuffle=False)
  
  # Create a linear regressor object.
  my_optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
  my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)
  linear_regressor = tf.estimator.LinearRegressor(
      feature_columns=feature_columns,
      optimizer=my_optimizer
  )

  # Set up to plot the state of our model's line each period.
  plt.figure(figsize=(15, 6))
  plt.subplot(1, 2, 1)
  plt.title("Learned Line by Period")
  plt.ylabel(my_label)
  plt.xlabel(my_feature)
  sample = california_housing_dataframe.sample(n=300)
  plt.scatter(sample[my_feature], sample[my_label])
  colors = [cm.coolwarm(x) for x in np.linspace(-1, 1, periods)]

  # Train the model, but do so inside a loop so that we can periodically assess
  # loss metrics.
  print("Training model...")
  print("RMSE (on training data):")
  root_mean_squared_errors = []
  for period in range (0, periods):
    # Train the model, starting from the prior state.
    linear_regressor.train(
        input_fn=training_input_fn,
        steps=steps_per_period
    )
    # Take a break and compute predictions.
    predictions = linear_regressor.predict(input_fn=prediction_input_fn)
    predictions = np.array([item['predictions'][0] for item in predictions])
    
    # Compute loss.
    root_mean_squared_error = math.sqrt(
        metrics.mean_squared_error(predictions, targets))
    # Occasionally print the current loss.
    print("  period %02d : %0.2f" % (period, root_mean_squared_error))
    # Add the loss metrics from this period to our list.
    root_mean_squared_errors.append(root_mean_squared_error)
    # Finally, track the weights and biases over time.
    # Apply some math to ensure that the data and line are plotted neatly.
    y_extents = np.array([0, sample[my_label].max()])
    
    weight = linear_regressor.get_variable_value('linear/linear_model/%s/weights' % input_feature)[0]
    bias = linear_regressor.get_variable_value('linear/linear_model/bias_weights')

    x_extents = (y_extents - bias) / weight
    x_extents = np.maximum(np.minimum(x_extents,
                                      sample[my_feature].max()),
                           sample[my_feature].min())
    y_extents = weight * x_extents + bias
    plt.plot(x_extents, y_extents, color=colors[period]) 
  print("Model training finished.")

  # Output a graph of loss metrics over periods.
  plt.subplot(1, 2, 2)
  plt.ylabel('RMSE')
  plt.xlabel('Periods')
  plt.title("Root Mean Squared Error vs. Periods")
  plt.tight_layout()
  plt.plot(root_mean_squared_errors)

  # Output a table with calibration data.
  calibration_data = pd.DataFrame()
  calibration_data["predictions"] = pd.Series(predictions)
  calibration_data["targets"] = pd.Series(targets)
  display.display(calibration_data.describe())

  print("Final RMSE (on training data): %0.2f" % root_mean_squared_error)

## Tugas 1: Dapatkan nilai ARKG sebesar 180 atau Lebih Rendah

Sesuaikan hyperparameter model untuk meningkatkan kerugian dan sesuaikan distribusi target dengan lebih baik. Jika setelah 5 menit atau lebih, Anda mengalami kesulitan mencapai nilai ARKG sebesar 180, lihat solusi untuk kombinasi yang memungkinkan.

In [ ]:
train_model(
    learning_rate=0.00001,
    steps=100,
    batch_size=1
)

### Solusi

Klik di bawah untuk mendapatkan satu solusi yang memungkinkan.

In [ ]:
train_model(
    learning_rate=0.00002,
    steps=500,
    batch_size=5
)

Ini hanyalah satu konfigurasi yang memungkinkan; mungkin ada kombinasi setelan lainnya yang juga memberikan hasil yang baik. Perhatikan bahwa pada umumnya, latihan ini bukanlah tentang menemukan setelan *terbaik*, namun untuk membantu mengasah intuisi Anda tentang bagaimana penyesuaian terhadap konfigurasi model memengaruhi kualitas prediksi.

### Apakah Ada Heuristik Standar untuk Penyesuaian Model?

Pertanyaan ini sering diajukan. Jawaban singkatnya, efek dari hyperparameter yang berbeda bergantung pada data. Sehingga tidak ada aturan yang tetap dan pasti; Anda harus menguji data tersebut.

Dengan demikian, berikut adalah beberapa aturan praktis yang dapat membantu memandu Anda:

 * Error pelatihan harus berkurang dengan stabil, tajam di awal, namun akhirnya harus berkurang dengan stabil seiring konvergensi pelatihan.
 * Jika pelatihan tidak dikonvergensi, coba jalankan dalam durasi yang lebih lama.
 * Jika error pelatihan berkurang sangat lambat, meningkatkan kecepatan pembelajaran dapat membantu menguranginya dengan lebih cepat.
   * Namun terkadang hal sebaliknya dapat terjadi jika kecepatan pembelajaran terlalu tinggi.
 * Jika error pelatihan sangat bervariasi, cobalah mengurangi kecepatan pembelajaran.
   * Kecepatan pembelajaran yang lebih rendah, ditambah dengan jumlah langkah yang lebih besar atau ukuran tumpukan yang lebih besar, sering kali menjadi kombinasi yang baik.
 * Ukuran tumpukan yang sangat kecil juga dapat menyebabkan ketidakstabilan. Pertama, cobalah nilai yang lebih besar seperti 100 atau 1.000, lalu turunkan hingga Anda melihat degradasi.

Sekali lagi, jangan terlalu mengikuti aturan praktis ini, karena efeknya tergantung pada data. Selalu lakukan uji coba dan verifikasi.

## Tugas 2: Cobalah Fitur yang Berbeda

Lihat apakah Anda dapat melakukannya lebih baik dengan mengganti fitur `total_rooms` dengan fitur `population`.

Jangan menghabiskan waktu lebih dari 5 menit untuk melakukan tindakan ini.

In [ ]:
# YOUR CODE HERE

### Solusi

Klik di bawah untuk mendapatkan satu solusi yang memungkinkan.

In [ ]:
train_model(
    learning_rate=0.00002,
    steps=1000,
    batch_size=5,
    input_feature="population"
)